In [85]:
using Laplacians
using SuiteSparse
using LinearAlgebra
using SparseArrays
using Statistics

┌ Info: Precompiling Laplacians [6f8e5838-0efe-5de0-80a3-5fb4f8dbb1de]
└ @ Base loading.jl:1260
Entropy pool not available to seed RNG; using ad-hoc entropy sources.
Entropy pool not available to seed RNG; using ad-hoc entropy sources.


In [86]:
import Laplacians.LLMatOrd
import Laplacians.approxChol
import Laplacians.LDLsolver
import Laplacians.approxchol_lapGiven
import Laplacians.LLord
import Laplacians.LDLinv
import Laplacians.LLcol

In [87]:
s=Set()
t = tuple(1,2)
println(t[1])
push!(s, tuple(1,2))
println(s)
pop!(s,tuple(1,2))
println(s)
for i in 1:3
    print(i)
end
rand(1:5)
!(2 in [])

1
Set(Any[(1, 2)])
Set(Any[])
123

true

In [181]:
function generate(n::Integer, m::Integer, uniform::Bool)
    s = Set(2:n)
    t = Set(1)
    edges = Set()
    cur_vertex = 1;
    while  !(isempty(s))
        num = rand(1:n)
        if (num in s)
            pop!(s, num)
            push!(t, num)
            push!(edges, tuple(cur_vertex, num))
        end
        cur_vertex = num
    end
    
    for i in 1:m-n+1
        u = rand(1:n)
        v = rand(1:n)
        while tuple(u,v) in edges || tuple(v,u) in edges || u==v
            u = rand(1:n)
            v = rand(1:n)
        end
        push!(edges, tuple(u,v))
    end
    
    u=[];v=[];w=[]
    for edge in edges
        if uniform
            push!(w, 1.0, 1.0)
        else
            weight = rand(Float64)
            push!(w, weight, weight)
        end
        push!(u, edge[1], edge[2])
        push!(v, edge[2], edge[1])
    end
#     println(u,v,w)
#     println(typeof(w))
    mat = sparse(u,v,w,n,n)
    return convert(SparseMatrixCSC{Float64, Int64}, mat)
end

generate (generic function with 1 method)

In [186]:
a = generate(1000, 5000, true)
la = lap(a)

1000×1000 SparseMatrixCSC{Float64,Int64} with 11000 stored entries:
  [1   ,    1]  =  9.0
  [115 ,    1]  =  -1.0
  [144 ,    1]  =  -1.0
  [186 ,    1]  =  -1.0
  [197 ,    1]  =  -1.0
  [392 ,    1]  =  -1.0
  [529 ,    1]  =  -1.0
  [575 ,    1]  =  -1.0
  [608 ,    1]  =  -1.0
  [731 ,    1]  =  -1.0
  [2   ,    2]  =  10.0
  [70  ,    2]  =  -1.0
  ⋮
  [905 ,  999]  =  -1.0
  [917 ,  999]  =  -1.0
  [998 ,  999]  =  -1.0
  [999 ,  999]  =  7.0
  [183 , 1000]  =  -1.0
  [210 , 1000]  =  -1.0
  [233 , 1000]  =  -1.0
  [457 , 1000]  =  -1.0
  [746 , 1000]  =  -1.0
  [751 , 1000]  =  -1.0
  [842 , 1000]  =  -1.0
  [913 , 1000]  =  -1.0
  [1000, 1000]  =  8.0

In [70]:
n = size(a)[1]
b = randn(size(la,1))
b = b.- mean(b)

4-element Array{Float64,1}:
 -0.5270249460796002
  0.7282324157897637
  0.3227082222861411
 -0.5239156919963046

In [73]:
a*b

4-element Array{Float64,1}:
  1.0509406380759048
 -1.0509406380759048
 -1.0509406380759048
  1.0509406380759048

In [71]:
llmat = LLMatOrd(a)
ldli = approxChol(llmat)
ldli

LDLinv{Int64,Float64}([1, 2, 3], [1, 3, 5, 6], [2, 3, 3, 4, 4], [0.5, 1.0, 0.3333333333333333, 1.0, 1.0], [0.5, 0.6666666666666667, 1.3333333333333335, 0.0])

In [72]:
import Laplacians.pcg 
pcg(la, b, F; tol=1e-20, maxits=1000, maxtime=Inf, verbose=true, pcgIts=Int[],stag_test=0)

PCG Stopped due to small or large rho
PCG stopped after: 0.0 seconds and 1 iterations with relative error 5.095756488149852e-17.


4-element Array{Float64,1}:
 -0.13214489328031198
  0.23274862813539376
  0.029986531383582427
 -0.13059026623866424

In [61]:
p = a*F(b)
p

9-element Array{Float64,1}:
  0.17417167277113102
  0.21542756396164142
 -0.31862173863635235
 -0.6734530603397146
 -0.5634301539201645
  1.2466318699958443
  1.3664517615312985
 -1.4492403981543465
  0.0020624827906623594

In [54]:
solver = approxchol_lapGiven(a); 
y_p = solver(b)
y_p

4-element Array{Float64,1}:
 -0.18704434899796174
 -0.3329585606514064
  0.3847296609402301
  0.13527324870913796

In [ ]:
norm(la*y-b)/norm(b) 

In [ ]:
norm(la*y_p-b)/norm(b)